# Worksheet 05

Name: Ziye Chen  
UID: U98411098 

### Topics

- Cost Functions
- Kmeans

### Cost Function

Solving Data Science problems often starts by defining a metric with which to evaluate solutions were you able to find some. This metric is called a cost function. Data Science then backtracks and tries to find a process / algorithm to find solutions that can optimize for that cost function.

For example suppose you are asked to cluster three points A, B, C into two non-empty clusters. If someone gave you the solution `{A, B}, {C}`, how would you evaluate that this is a good solution?

Notice that because the clusters need to be non-empty and all points must be assigned to a cluster, it must be that two of the three points will be together in one cluster and the third will be alone in the other cluster.

In the above solution, if A and B are closer than A and C, and B and C, then this is a good solution. The smaller the distance between the two points in the same cluster (here A and B), the better the solution. So we can define our cost function to be that distance (between A and B here)!

The algorithm / process would involve clustering together the two closest points and put the third in its own cluster. This process optimizes for that cost function because no other pair of points could have a lower distance (although it could equal it).

### K means

a) (1-dimensional clustering) Walk through Lloyd's algorithm step by step on the following dataset:

`[0, .5, 1.5, 2, 6, 6.5, 7]` (note: each of these are 1-dimensional data points)

Given the initial centroids:

`[0, 2]`

Step 1:
- Label 0 (closer to 0) : `[0, .5]`
- Label 1 (closer to 2) : `[1.5, 2, 6, 6.5, 7]`

centroids: $0 \to 0.25$, $2 \to 4.6$

Step 2:
- Label 1 (closer to 0.25) : `[0, 0.5, 1.5, 2]`
- Label 1 (closer to 4.6) : `[6, 6.5, 7]`

centroids: $0.25 \to 1$, $4.6 \to 6.5$

Step 3 (Final Step):
- Label 1 (closer to 1) : `[0, 0.5, 1.5, 2]`
- Label 1 (closer to 6.5) : `[6, 6.5, 7]`

centroids: $1$, $6.5$

b) Describe in plain english what the cost function for k means is.

The cost function for k means calculates how far each data point is from the centroid of its assigned cluster and then sums up these distances. 

c) For the same number of clusters K, why could there be very different solutions to the K means algorithm on a given dataset?

The k-means algorithm starts with a random selection of centroids. The initial centroids can affect the final clustering outcome. If the initial centroids are not well-positioned, they might lead to local minima.

d) Does Lloyd's Algorithm always converge? Why / why not?

Yes. Lloyd's Algorithm always converges. Cost function decrease each time data is clustered in next step. If the algorithm reaches a point where cost function no longer decreases, it has converged.

e) Follow along in class the implementation of Kmeans

In [6]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

centers = [[0, 0], [2, 2], [-3, 2], [2, -4]]
X, _ = datasets.make_blobs(n_samples=300, centers=centers, cluster_std=1, random_state=0)

class KMeans():

    def __init__(self, data, k):
        self.data = data
        self.k = k
        self.assignment = [-1 for _ in range(len(data))]
        self.snaps = []
    
    def snap(self, centers):
        TEMPFILE = "temp.png"

        fig, ax = plt.subplots()
        ax.scatter(X[:, 0], X[:, 1], c=self.assignment)
        ax.scatter(centers[:, 0], centers[:, 1], c='r')
        fig.savefig(TEMPFILE)
        plt.close()
        self.snaps.append(im.fromarray(np.asarray(im.open(TEMPFILE))))


    def initialize(self):
        return self.data[np.random.choice(range(len(self.data)), self.k)]

    def distance(self, x, y):
        return np.linalg.norm(abs(x - y))

    def assign(self, centers):
        for i in range(len(self.data)):
            minimum = self.distance(centers[0], self.data[i])
            self.assignment[i] = 0
            for j in range(1, len(centers)):
                dist = self.distance(centers[j], self.data[i])
                if dist < minimum:
                    self.assignment[i] = j
                    minimum = dist
        return
    
    def is_diff_clusters(self, centers, new_centers):
        for i in range(len(centers)):
            if self.distance(centers[i], new_centers[i]) != 0:
                return True
        return False
    
    def get_centers(self):
        new_centers = []
        for i in set(self.assignment):
            cluster = []
            for j in range(len(self.data)):
                if self.assignment[j] == i:
                    cluster.append(self.data[j])
            if cluster:
                new_centers.append(np.mean(cluster, axis=0))
        return np.array(new_centers)
            
    
    def lloyds(self):
        centers = self.initialize()
        self.assign(centers)
        self.snap(centers)
        new_centers = self.get_centers()
        while self.is_diff_clusters(centers, new_centers):
            self.assign(new_centers)
            centers = new_centers
            self.snap(new_centers)
            new_centers = self.get_centers()
            
        
        return

            

kmeans = KMeans(X, 6)
kmeans.lloyds()
images = kmeans.snaps

images[0].save(
    'kmeans.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=500
)